All quotes below are excerpted from https://toadstyle.org/cryptopals/60.txt

---
```
61. Duplicate-Signature Key Selection in ECDSA (and RSA)

Suppose you have a message-signature pair. If I give you a public key
that verifies the signature, can you trust that I'm the author?

You shouldn't. It turns out to be pretty easy to solve this problem
across a variety of digital signature schemes. If you have a little
flexibility in choosing your public key, that is.

Let's consider the case of ECDSA.

First, implement ECDSA. If you still have your old DSA implementation
lying around, this should be straightforward. All the same, here's a
refresher if you need it:

    function sign(m, d):
        k := random_scalar(1, n)
        r := (k * G).x
        s := (H(m) + d*r) * k^-1
        return (r, s)

    function verify(m, (r, s), Q):
        u1 := H(m) * s^-1
        u2 := r * s^-1
        R := u1*G + u2*Q
        return r = R.x

Remember that all the scalar operations are mod n, the order of the
base point G. (d, Q) is the signer's key pair. H(m) is a hash of the
message.

Note that the verification function requires arbitrary point
addition. This means your Montgomery ladder (which only performs
scalar multiplication) won't work here. This is no big deal; just fall
back to your old Weierstrass imlpementation.
```

In [1]:
from challenge_59 import Curve  # Weierstrass curve

from dataclasses import dataclass
from hashlib import sha256
from random import SystemRandom
randrange = SystemRandom().randrange  # we'll use a CSPRNG-backed randrange instead of the (insecure) default

In [2]:
# parameters from cryptopals 59:
curve = Curve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)
base = (182, 85518893674295321206118380980485522083)
order = 29246302889428143187362802287225875743

In [3]:
@dataclass
class ECDSA:
    curve: Curve
    base: tuple
    order: int

    def sign(self, m: bytes, d: int):
        curve, G, order = self.curve, self.base, self.order
        hm = int(sha256(m).hexdigest(), base=16)

        while True:  # loop for retrying k based on s
            while True:  # loop for retrying k based on r
                k = randrange(1, order)
                r = curve.mul(G, k)[0] % order
                if r != 0: break
            s = (pow(k, -1, order) * (hm + d*r)) % order
            if s != 0: break

        return (r, s)

    def verify(self, m, sig, pubkey):
        r, s = sig
        order = self.order
        if not (0 < r < order and 0 < s < order):
            print("ERR: r or s out of bounds!")
            return False
        curve, G = self.curve, self.base
        s_inv = pow(s, -1, order)
        hm = int(sha256(m).hexdigest(), base=16)
        u1 = (hm * s_inv) % order
        u2 = (r * s_inv) % order
        R = curve.add(curve.mul(base, u1),
                      curve.mul(pubkey, u2))
        return r == R[0] % order

    def new_keypair(self):
        d = randrange(1, order)
        Q = self.curve.mul(self.base, d)
        return (d, Q)  # private, public

In [9]:
dsa = ECDSA(curve, base, order)
alice = dsa.new_keypair()
_alice_priv, alice_pub = alice
m = b"they tell me i don't have a long time to change your mind"

sig = dsa.sign(m, _alice_priv)
print(sig)
print(dsa.verify(m, sig, alice_pub))

(12029088164824484183368164751430187090, 22178218742078463887332301608157236967)
True
